<a href="https://colab.research.google.com/github/stevengregori92/Maximize-Titanic-with-Jcopml-and-Sklearn/blob/main/Maximize_Titanic_with_Jcopml_and_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install luwiji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.0 MB/s eta 0:00:00
  Created wheel for luwiji: filename=luwiji-1.2.1-py2.py3-none-any.whl size=40366842 sha256=c932830eda8a9a98e6ebbc01187d45012dad79e305d8ffd4de679e84af299e8f
  Stored in directory: /root/.cache/pip/wheels/1b/49/54/8dfa7de82599da47df8c290f769ade64075dc7a0a2a4fd7414
  Created wheel for jcopml: filename=jcopml-1.2.2-py2.py3-none-any.whl size=32194 sha256=95db2bc96ab1bd42bce088e40e32f138e4d5bef84c6e65441157f5d124e3889a
  Stored in directory: /root/.cache/pip/wheels/a6/1c/91/32f7a6b0a9d6bbcef17a7fc0d065f9a9b8d9904551b377b64d
Successfully built luwiji jcopml


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.tuning import grid_search_params as gsp

df = pd.read_csv('titanic.csv', index_col='PassengerId')
df.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)

pd.crosstab(df.Pclass, df.Survived, normalize='index')

df['isAlone'] = (df.SibSp==0) & (df.Parch ==0)
df.Age = pd.cut(df.Age, [0,5,12,18,40,120], labels=['balita', 'anak-anak', 'remaja', 'dewasa', 'lansia'])
df.Fare = pd.cut(df.Fare, [0,25,50,75,600], labels=['murah', 'normal','mahal', 'eksetkutif'])

# Encoding all categorical columns
df = pd.get_dummies(df, columns=['Pclass', 'Sex', 'Embarked', 'Age', 'Fare', 'isAlone'])

# Data Splitting
X = df.drop(columns=['Survived'])
y = df.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Preprocessing
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(scaling='minmax'), ['Parch', 'SibSp']),
    ('categoric', cat_pipe(encoder='onehot'), X.select_dtypes(include='uint8').columns)
])

# Training
pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', KNeighborsClassifier())
])

model = GridSearchCV(pipeline, gsp.knn_params, cv=3, n_jobs=-1, verbose=1, error_score='raise')
model.fit(X_train, y_train)

# Evaluation
print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 90 candidates, totalling 270 fits
{'algo__n_neighbors': 5, 'algo__p': 2, 'algo__weights': 'uniform'}
0.8469101123595506 0.8216442695221549 0.8100558659217877
